In [11]:
contents = ["大巨蛋案對市府同仁下封口令？柯P否認", "柯P市府近來飽受大巨蛋爭議"]

In [19]:
import jieba
jieba.load_userdict('userdict.txt')
corpus = []
for sentence in contents:
    #print(list(jieba.cut(sentence)))
    corpus.append(' '.join(jieba.cut(sentence)))

In [20]:
corpus

['大巨蛋 案對 市府 同仁 下 封口令 ？ 柯P 否認', '柯P 市府 近來 飽受 大巨蛋 爭議']

In [23]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
X

<2x10 sparse matrix of type '<class 'numpy.longlong'>'
	with 13 stored elements in Compressed Sparse Row format>

In [25]:
print(vectorizer.get_feature_names())

['同仁', '否認', '大巨蛋', '封口令', '市府', '柯p', '案對', '爭議', '近來', '飽受']


In [24]:
X.toarray()

array([[1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
       [0, 0, 1, 0, 1, 1, 0, 1, 1, 1]], dtype=int64)

In [26]:
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
cosine_similarity(X)

array([[1.        , 0.46291005],
       [0.46291005, 1.        ]])

In [27]:
cosine_distances(X)

array([[0.        , 0.53708995],
       [0.53708995, 0.        ]])

## 根據新聞相似度推薦新聞

In [30]:
import pandas
news = pandas.read_excel('https://raw.githubusercontent.com/ywchiu/taipeifubon/master/data/20171214news.xlsx', index_col = 0)

In [36]:
import jieba
jieba.load_userdict('userdict.txt')
corpus = []
titles = []
for idx, rec in news.iterrows():
    corpus.append(' '.join(jieba.cut(rec.get('content'))))
    titles.append(rec.get('title'))

In [38]:
#corpus[0]
titles[0]

'【更新】水利會改官派明闖關\u3000綠委24小時前顧議場大門防藍突襲'

In [40]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

In [41]:
X

<899x28957 sparse matrix of type '<class 'numpy.longlong'>'
	with 122298 stored elements in Compressed Sparse Row format>

In [43]:
from sklearn.metrics.pairwise import cosine_distances
cs = cosine_distances(X)
cs.shape

(899, 899)

In [46]:
titles[12]

'黃國昌：長期目標要消滅國民黨\u3000不支持柯文哲的兩岸一家親'

In [48]:
cs[12].shape

(899,)

In [52]:
import numpy 
students = [60,30,40,70,90,80,]
students_ary = numpy.array(students)
students_ary.argsort()

array([1, 2, 0, 3, 5, 4])

In [56]:
for idx in cs[12].argsort()[0:10]:
    print(titles[idx], cs[12][idx])

黃國昌：長期目標要消滅國民黨　不支持柯文哲的兩岸一家親 0.0
黃國昌若被罷免會更強　他：成為選輸北市的阿扁 0.5542565596535691
罷免案將投票　李遠哲今再度現身力挺黃國昌 0.5752596978859429
「兩蔣時代」超譯　網友：不准傷皇城內的和氣！ 0.5938835126582123
罷昌案周六投票　時代力量全力澄清不實謠言 0.6402804718905146
【聲援片】沈發惠呼籲罷昌案投「不同意」　黃國昌：謝謝您 0.7003246342755608
宗教團體可曾想過：黃國昌犀利形象恐越打越旺 0.7749027801375493
【歷史上的今天】幫黃國昌站台　蔡英文：我有準備當總統了 0.7803630780883579
【更新】水利會改官派明闖關　綠委24小時前顧議場大門防藍突襲 0.7869878365124581
臉書在台灣繳稅太少　財政部將進一步追稅 0.7926505336912069


In [58]:
pos = 12
print('查詢文章:',titles[pos])
for idx in cs[pos].argsort()[1:10]:
    if cs[pos][idx] < 0.7:
        print('相關文章',titles[idx], cs[pos][idx])

查詢文章: 黃國昌：長期目標要消滅國民黨　不支持柯文哲的兩岸一家親
相關文章 黃國昌若被罷免會更強　他：成為選輸北市的阿扁 0.5542565596535691
相關文章 罷免案將投票　李遠哲今再度現身力挺黃國昌 0.5752596978859429
相關文章 「兩蔣時代」超譯　網友：不准傷皇城內的和氣！ 0.5938835126582123
相關文章 罷昌案周六投票　時代力量全力澄清不實謠言 0.6402804718905146


In [59]:
def getSimiliarNews(pos):
    print('查詢文章:',titles[pos])
    for idx in cs[pos].argsort()[1:10]:
        if cs[pos][idx] < 0.7:
            print('相關文章',titles[idx], cs[pos][idx])

In [65]:
getSimiliarNews(35)

查詢文章: FBI探員罵川普「白痴」　遭調離通俄門調查
相關文章 【公道難伸張】再控川普性騷　受害者：沒人在乎很受傷 0.6434278035355978
相關文章 老謝：美菲2位狂人兄弟 今年交出不錯成績單　 0.6502913209540035
相關文章 否認性騷指控　川普：民主黨編的假新聞 0.6754198672772045
相關文章 諷參議員陸天娜「為錢啥都做」　川普遭美媒批不適任總統 0.6984356814409867


In [67]:
q = '黃國昌 李遠哲'
q_list = [' '.join(jieba.cut(q))]

In [71]:
q_vec = vectorizer.transform(q_list)
q_vec

<1x28957 sparse matrix of type '<class 'numpy.int64'>'
	with 2 stored elements in Compressed Sparse Row format>

In [73]:
q_cs = cosine_distances(q_vec, X)

In [74]:
q_cs.shape

(1, 899)

In [79]:
q_cs.argsort()[0][0]

182

In [80]:
titles[182]

'黃國昌若被罷免會更強\u3000他：成為選輸北市的阿扁'

## 小任務(一)
- 請先寫個網路爬蟲爬取台北富邦銀行的常見問題，並將問題與答案整理名為qa的DataFrame？
- https://www.fubon.com/banking/faq/index.htm

In [83]:
import requests
from bs4 import BeautifulSoup
res = requests.get('https://www.fubon.com/banking/FAQ_Data//faq/index_data/faqData1.xml')
soup = BeautifulSoup(res.text, 'lxml')

In [89]:
qa_ary = []
for item in soup.select('item'):
    question = item.select_one('title').text
    answer_tmp = item.select_one('description').text
    answer = BeautifulSoup(answer_tmp, 'lxml')
    dic = {'question': question, 'answer': answer.text}
    qa_ary.append(dic)

In [90]:
import pandas
qa = pandas.DataFrame(qa_ary)
qa.head()

,answer,question
0,感謝您申辦本行信用卡，如您尚需補件，請您備妥相關資料並在文件上註明「身分證字號」、「聯絡電話...,申請信用卡有缺少資料應如何補件？
1,只要是作業系統為 Android 5.0以上且內建NFC功能的手機即可使用。為確保交易穩定性...,什麼型號的手機可支援HCE行動信用卡？
2,請確認您輸入的註冊手機門號是否有誤、是否有設定阻擋企業簡訊功能。,註冊「台灣行動支付」APP時沒有收到驗證碼，請問怎麼辦？
3,可以，但若您留存於本行的手機門號並非目前實際使用的門號，請務必致電本行客服中心更新您的手機門...,註冊「台灣行動支付」APP之手機門號可以和留存於銀行之手機門號不同嗎？
4,您可透過拍攝或手動輸入您欲下載的卡號、有效期限、卡片背面簽名欄後3碼（實體卡片需先完成開卡）...,如何將卡片新增於我的手機上？


In [91]:
len(qa)

231

## 小任務 (二)
- 試著建立一個依內容相似度回應答案的問答機器人，當使用者輸入問題時，機器人可以依相似問題給予對應答案


In [96]:
import jieba
jieba.load_userdict('/Users/davidchiu/Desktop/userdict2.txt')

### 根據 qa DataFrame建立詞頻矩陣

In [113]:
import jieba
jieba.load_userdict('userdict.txt')
corpus = []
answers = []
for idx, rec in qa.iterrows():
    corpus.append(' '.join(jieba.cut(rec.get('question'))))
    answers.append(rec.get('answer'))

### 使用sklearn 的CountVectorizer 建立詞頻矩陣

In [114]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

In [115]:
X

<231x559 sparse matrix of type '<class 'numpy.longlong'>'
	with 1971 stored elements in Compressed Sparse Row format>

### 把使用者輸入的問題變成是詞向量

In [117]:
user_query = input('請輸入你的問題:')
user_query_list = [' '.join(jieba.cut(user_query))]

請輸入你的問題:如何計算就學貸款額度?


In [118]:
user_query_list

['如何 計算 就學 貸款 額度 ?']

In [119]:
user_query_vec = vectorizer.transform(user_query_list)

In [120]:
user_query_vec

<1x559 sparse matrix of type '<class 'numpy.int64'>'
	with 5 stored elements in Compressed Sparse Row format>

### 根據使用者輸入的問題給予最相近的答案

In [121]:
from sklearn.metrics.pairwise import cosine_distances
cs  = cosine_distances(user_query_vec, X)

In [122]:
cs.shape

(1, 231)

In [123]:
corpus[cs.argsort()[0][0]]

'就學 貸款 之 額度 如何 計算 ？'

In [124]:
answers[cs.argsort()[0][0]]

'1.學生申請就學貸款之金額，以下列各項費用為範圍：\n\n(1.)學雜費：其金額為該學期實際繳納者。【另依據教育部93 年7 月29 日台高（四）字第0930097146 號函釋：鑒於「音樂指導費」為音樂系學生應繳納費用之一，故得納入學雜費貸款金額項下。】\n(2.)實習費：其金額為該學期實際繳納者。\n(3.)書籍費：高級中等學校為每生每學期新臺幣1,000 元；專科以上學校為每生每學期新臺幣3,000 元。\n(4.)住宿費：其金額為該校住校宿舍費，校外住宿學生申貸之住宿費，以該校住校宿舍費之最高者為基準。\n(5.)學生團體保險費：其金額為該學期實際繳納者。\n(6.)海外研修費：每生每年以新臺幣44 萬元為上限。(需檢附蓋有學校戳章之「申請就學貸款海外研修費證明」)。\n(7.)生活費：低收入戶學生，每生每學期以新臺幣4 萬元為上限；中低收入戶學生，每生每學期以新臺幣2 萬元為上限。生活費貸款者，應為經直轄市、縣（市）社政主管機關核定為低收入戶或中低收入戶之學生。\n(8.)電腦及網路通訊使用費：其金額依該管主管機關之規定。\n\n2.享受全部公費之公費生或已獲得政府主辦之其他無息助學貸款者，不得申請就學貸款。但辦理學雜費減免或已請領教育部助學金之學生，得就學雜等各費減除學雜費減免或教育部助學金後之差額申請就學貸款。\n3.書籍費、校外住宿費或生活費由學校逕行發給學生。\n4.就學貸款原則採「每一教育階段學程簽訂一張總額度借據」方式辦理。每一教育階段學程總額度，係由本行依據第1 點所列7 個可申貸項目，估算每一教育階段學程所需花費之總金額而訂定（例如高中階段是估算6 個學期所需花費之金額），並一體適用於各該教育階段之申請人。\n5.借據所定借款額度如因學生在國內就學時間延長或學雜各費調漲等事由而致不敷使用者，依借據之約定，本行得於知悉該事由時，以「借款額度調整通知書」通知學生及連帶保證人調整後之額度，學生及其連帶保證人則應就實際申請撥款之合計金額負還款責任。'

In [126]:
def getAnswer(user_query):
    user_query_list = [' '.join(jieba.cut(user_query))]    
    user_query_vec = vectorizer.transform(user_query_list)
    cs  = cosine_distances(user_query_vec, X)
    return answers[cs.argsort()[0][0]]

In [128]:
user_query = input('請輸入你的問題:')
print(getAnswer(user_query))

請輸入你的問題:請問申請信用貸款會有哪些費用？
本行信用貸款僅收取帳戶管理費與匯款手續費，上述費用將於撥款時自借款金額中扣除，除此之外並無任何其他費用，撥貸前亦不會跟您收取任何費用。若您辦理信用貸款時，遭要求「先匯款」或「收取現金」者，應屬代辦公司業務招攬方式，請勿向其申辦，請直接與申辦專線0809-01-02-03聯繫申辦。
